# Korf's IDA* - Optimal Solving with Pattern Databases

**Author:** Alex Toska  
**Affiliation:** University of Patras  
**Algorithm:** Korf's IDA* (1997)  

---

## Overview

Richard Korf's algorithm finds **optimal solutions** to the Rubik's Cube using IDA* (Iterative Deepening A*) with pattern databases.

This notebook covers:
- IDA* algorithm and heuristics
- Pattern database construction
- Admissible heuristics
- Optimal solution finding
- Time-memory trade-offs

## Setup

In [ ]:
import sys
from pathlib import Path
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.cube.rubik_cube import RubikCube
from src.korf.solver import KorfSolver
from src.cube.visualization import display_cube_unfolded
from src.cube.visualize_3d import visualize_cube_3d

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 5)

print("✓ Imports successful!")

## 1. IDA* Algorithm

### What is IDA*?

**IDA*** (Iterative Deepening A*) combines:
- **Depth-first search** - Memory efficient
- **A* heuristic** - Optimal solutions
- **Iterative deepening** - Guaranteed to find shortest path

### Algorithm Pseudocode

```
function IDA*(root, goal):
    bound = h(root)  // Initial heuristic estimate
    
    while True:
        result = search(root, 0, bound)
        
        if result == FOUND:
            return solution
        if result == INFINITY:
            return NO_SOLUTION
        
        bound = result  // Next depth threshold

function search(node, g, bound):
    f = g + h(node)  // f = cost so far + heuristic
    
    if f > bound:
        return f  // Exceeded bound, try deeper
    
    if is_goal(node):
        return FOUND
    
    min_bound = INFINITY
    for successor in expand(node):
        result = search(successor, g+1, bound)
        if result == FOUND:
            return FOUND
        min_bound = min(min_bound, result)
    
    return min_bound
```

### Key Properties

- **Optimal:** Always finds shortest solution
- **Complete:** Will find solution if one exists
- **Memory efficient:** O(depth) vs O(states) for A*
- **Requires admissible heuristic:** h(n) ≤ true distance

## 2. Pattern Databases

### What are Pattern Databases?

Pattern databases pre-compute the **exact distance** from any pattern configuration to the solved state.

### Korf's Three Pattern Databases

#### 1. Corner Pattern Database
- **Size:** 8! × 3⁷ = 88,179,840 states (~88 million)
- **Memory:** ~88 MB (1 byte per state)
- **Covers:** All 8 corners (position + orientation)
- **Ignores:** All edges

#### 2. Edge Group 1 Database
- **Size:** 12!/(12-6)! × 2⁶ = 42,577,920 states (~43 million)
- **Memory:** ~43 MB
- **Covers:** 6 edges (e.g., UF, UR, UB, UL, DF, DB)
- **Ignores:** Other 6 edges, all corners

#### 3. Edge Group 2 Database
- **Size:** 12!/(12-6)! × 2⁶ = 42,577,920 states (~43 million)
- **Memory:** ~43 MB
- **Covers:** Other 6 edges (e.g., FR, FL, BR, BL, DR, DL)
- **Ignores:** First 6 edges, all corners

**Total Memory:** ~174 MB (but typically ~45 MB with compression)

### Admissible Heuristic

```
h(state) = max(
    corner_db[corners(state)],
    edge1_db[edges1(state)],
    edge2_db[edges2(state)]
)
```

This heuristic is:
- **Admissible:** Never overestimates (h ≤ true distance)
- **Consistent:** h(n) ≤ h(n') + cost(n, n')
- **Tight:** Close to true distance

Taking the **maximum** ensures admissibility while being as informed as possible.

## 3. Solving Example

In [ ]:
# Create and scramble a cube (shallow scramble for demo)
cube = RubikCube()
scramble = cube.scramble(moves=7, seed=42)  # Shallow for reasonable solve time

print("=== SCRAMBLE ===")
print(f"Scramble: {' '.join(scramble)}")
print(f"Scramble length: {len(scramble)} moves\n")

# Visualize
fig = visualize_cube_3d(cube, view_angles=(30, 45))
fig.suptitle("Scrambled Cube (7 moves)", fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Solve using Korf's IDA*
print("=== SOLVING WITH KORF'S IDA* ===")
print("Note: This may take several seconds...\n")

solver = KorfSolver(max_depth=20)

start_time = time.time()
solution = solver.solve(cube)
end_time = time.time()

print(f"\n✓ OPTIMAL solution found!")
print(f"Solution: {' '.join(solution)}")
print(f"Solution length: {len(solution)} moves")
print(f"Time: {end_time - start_time:.4f} seconds")

# Verify
test_cube = cube.copy()
for move in solution:
    test_cube.apply_move(move)

print(f"\nVerification: Cube is {'SOLVED ✓' if test_cube.is_solved() else 'NOT SOLVED ✗'}")
print(f"\nThis is an OPTIMAL solution (shortest possible)!")

In [ ]:
# Visualize solved cube
fig = visualize_cube_3d(test_cube, view_angles=(30, 45))
fig.suptitle("Solved Cube (Optimal Solution)", fontsize=14, fontweight='bold')
plt.show()

## 4. Time Complexity Analysis

### Search Space

For a depth-d optimal solution:
- **Nodes expanded:** ~18^d (branching factor ≈ 18)
- **With heuristic:** Significantly pruned

### Typical Performance

| Scramble Depth | Optimal Solution | Nodes Expanded | Time |
|----------------|------------------|----------------|------|
| 5 moves | 5 moves | ~10³-10⁴ | <0.1s |
| 7 moves | 7 moves | ~10⁴-10⁵ | 0.1-1s |
| 10 moves | 10 moves | ~10⁶-10⁷ | 1-10s |
| 15 moves | 15 moves | ~10⁹-10¹⁰ | 30-300s |
| 20 moves | 20 moves | ~10¹² | Hours-days |

### Why So Slow?

- **Exponential growth:** 18^d search space
- **Optimality guarantee:** Must explore all possibilities
- **God's Number:** Max depth is 20 moves

### Pattern Database Impact

Without pattern databases:
- Only solvable to ~12 moves in reasonable time

With pattern databases:
- Can solve up to ~15 moves in seconds
- Can solve up to ~18 moves in minutes
- Can solve any position theoretically (given enough time)

## 5. Performance Benchmark

**Warning:** This benchmark uses shallow scrambles (7 moves) to keep runtime reasonable.

In [ ]:
# Benchmark on shallow scrambles
print("Running benchmark on 5 scrambles (7 moves each)...\n")
print("Note: Deeper scrambles would take much longer!\n")

results = []
for seed in range(42, 47):  # Only 5 tests to keep runtime reasonable
    cube = RubikCube()
    scramble = cube.scramble(moves=7, seed=seed)
    
    solver = KorfSolver(max_depth=20)
    start = time.time()
    solution = solver.solve(cube)
    elapsed = time.time() - start
    
    results.append({
        'seed': seed,
        'scramble_length': len(scramble),
        'solution_length': len(solution),
        'time_seconds': elapsed
    })
    
    print(f"Seed {seed}: {len(solution)} moves (optimal) in {elapsed:.3f}s")

# Create DataFrame
df = pd.DataFrame(results)

print("\n=== Summary Statistics ===")
print(f"Average solution length: {df['solution_length'].mean():.1f} moves")
print(f"Min solution length: {df['solution_length'].min()} moves")
print(f"Max solution length: {df['solution_length'].max()} moves")
print(f"Average time: {df['time_seconds'].mean():.4f} seconds")
print(f"Min time: {df['time_seconds'].min():.4f} seconds")
print(f"Max time: {df['time_seconds'].max():.4f} seconds")
print("\nAll solutions are OPTIMAL (shortest possible)!")

In [ ]:
# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Solution lengths
ax1.bar(df['seed'] - 42, df['solution_length'], color='#9b59b6', alpha=0.7)
ax1.axhline(y=df['solution_length'].mean(), color='red', linestyle='--', 
            label=f"Mean: {df['solution_length'].mean():.1f}")
ax1.set_xlabel('Test Case', fontsize=12)
ax1.set_ylabel('Solution Length (moves)', fontsize=12)
ax1.set_title('Korf IDA* Solution Lengths (OPTIMAL)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Execution times
ax2.bar(df['seed'] - 42, df['time_seconds'], color='#e74c3c', alpha=0.7)
ax2.axhline(y=df['time_seconds'].mean(), color='red', linestyle='--',
            label=f"Mean: {df['time_seconds'].mean():.3f}s")
ax2.set_xlabel('Test Case', fontsize=12)
ax2.set_ylabel('Time (seconds)', fontsize=12)
ax2.set_title('Korf IDA* Execution Time', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Three-Way Comparison

In [ ]:
# Compare all three algorithms on same scramble
from src.thistlethwaite.solver import ThistlethwaiteSolver
from src.kociemba.solver import KociembaSolver

cube = RubikCube()
cube.scramble(moves=7, seed=100)  # Shallow for Korf

print("Comparing all three algorithms...\n")

# Thistlethwaite
t_solver = ThistlethwaiteSolver(use_pattern_databases=False)
t_start = time.time()
t_solution = t_solver.solve(cube.copy())
t_time = time.time() - t_start
print(f"Thistlethwaite: {len(t_solution)} moves in {t_time:.4f}s")

# Kociemba
k_solver = KociembaSolver(max_depth_phase1=12, max_depth_phase2=18)
k_start = time.time()
k_solution = k_solver.solve(cube.copy())
k_time = time.time() - k_start
print(f"Kociemba: {len(k_solution)} moves in {k_time:.4f}s")

# Korf
korf_solver = KorfSolver(max_depth=20)
korf_start = time.time()
korf_solution = korf_solver.solve(cube.copy())
korf_time = time.time() - korf_start
print(f"Korf IDA*: {len(korf_solution)} moves in {korf_time:.4f}s (OPTIMAL)")

# Display comparison table
comparison_data = [
    {
        "Algorithm": "Thistlethwaite",
        "Solution Length": len(t_solution),
        "Time (s)": f"{t_time:.4f}",
        "Optimal": "No"
    },
    {
        "Algorithm": "Kociemba",
        "Solution Length": len(k_solution),
        "Time (s)": f"{k_time:.4f}",
        "Optimal": "Near"
    },
    {
        "Algorithm": "Korf IDA*",
        "Solution Length": len(korf_solution),
        "Time (s)": f"{korf_time:.4f}",
        "Optimal": "Yes ✓"
    }
]

comp_df = pd.DataFrame(comparison_data)
print("\n=== Full Algorithm Comparison ===")
print(comp_df.to_string(index=False))

In [ ]:
# Visualize three-way comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

algorithms = ['Thistlethwaite', 'Kociemba', 'Korf IDA*']
moves = [len(t_solution), len(k_solution), len(korf_solution)]
times = [t_time, k_time, korf_time]
colors = ['#3498db', '#2ecc71', '#9b59b6']

# Solution length
bars1 = ax1.bar(algorithms, moves, color=colors, alpha=0.7)
ax1.set_ylabel('Moves', fontsize=12)
ax1.set_title('Solution Length Comparison', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars1):
    height = bar.get_height()
    label = f'{int(height)}'
    if i == 2:  # Korf
        label += ' ✓'
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             label, ha='center', va='bottom', fontsize=11, fontweight='bold' if i == 2 else 'normal')

# Time (log scale due to differences)
bars2 = ax2.bar(algorithms, times, color=colors, alpha=0.7)
ax2.set_ylabel('Seconds', fontsize=12)
ax2.set_title('Execution Time Comparison', fontsize=14, fontweight='bold')
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3, which='both')
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}s', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print(f"• Korf finds OPTIMAL solution: {len(korf_solution)} moves")
print(f"• Kociemba is close: {len(k_solution)} moves ({len(k_solution) - len(korf_solution)} extra)")
print(f"• Thistlethwaite uses most moves: {len(t_solution)} moves")
print(f"• Speed trade-off: Korf takes {korf_time/k_time:.1f}x longer than Kociemba")

## 7. Strengths & Weaknesses

### ✅ Strengths

1. **Optimal solutions:** Guaranteed shortest path (≤20 moves)
2. **Mathematically proven:** IDA* with admissible heuristic
3. **Memory efficient:** O(depth) vs O(states) for A*
4. **Pattern databases reusable:** Pre-compute once
5. **Research value:** Important theoretical contribution
6. **Educational:** Great for understanding search algorithms

### ❌ Weaknesses

1. **Very slow:** Can take seconds to minutes for deep scrambles
2. **Exponential time:** 18^d nodes in worst case
3. **Impractical for competition:** Too slow for speed solving
4. **Variable time:** Some positions take much longer
5. **Setup cost:** Pattern databases take time to build
6. **Depth sensitive:** Performance degrades rapidly with depth

### 🎯 Best Use Cases

- **Research** - Understanding optimal solving
- **Theoretical analysis** - Proving bounds
- **Benchmarking** - Comparing other algorithms
- **Education** - Teaching search algorithms
- **When optimality is critical** - Shortest solution required
- **Shallow positions** - ≤10 move scrambles

## 8. Time-Memory Trade-offs

### Pattern Database Size vs Search Time

| Strategy | Memory | Time | Solution |
|----------|--------|------|----------|
| **No databases** | ~1 MB | Very slow | Optimal |
| **Small databases** | ~10 MB | Slow | Optimal |
| **Korf's 3 databases** | ~45 MB | Medium | Optimal |
| **Large databases** | ~500 MB | Fast | Optimal |
| **Huge databases** | ~5 GB | Very fast | Optimal |

### Other Trade-offs

**Optimality vs Speed:**
- Korf: Optimal, slow
- Kociemba: Near-optimal, fast
- Thistlethwaite: Sub-optimal, very fast

**Memory vs Time:**
- More memory → Better heuristic → Faster search
- Less memory → Weaker heuristic → Slower search

**Depth vs Time:**
- Depth 7: ~0.1-1 second
- Depth 10: ~1-10 seconds
- Depth 15: ~30-300 seconds
- Depth 20: Hours to days

## 9. Implementation Details

### Project Structure

1. **Pattern Databases** (`src/korf/pattern_database.py`)
   - Corner database
   - Edge group 1 database
   - Edge group 2 database
   - BFS construction

2. **Solver** (`src/korf/solver.py`)
   - IDA* implementation
   - Heuristic calculation
   - Depth management
   - Move pruning

3. **Coordinates** (`src/korf/coordinates.py`)
   - Corner encoding
   - Edge encoding
   - Pattern extraction

### Usage Example

```python
from src.cube.rubik_cube import RubikCube
from src.korf.solver import KorfSolver

# Create and scramble
cube = RubikCube()
cube.scramble(moves=8, seed=42)

# Solve optimally
solver = KorfSolver(max_depth=20)
solution = solver.solve(cube)

print(f"Optimal solution: {' '.join(solution)}")
print(f"Length: {len(solution)} moves")
```

### Pattern Database Construction

```python
from src.korf.pattern_database import PatternDatabase

# Build corner database
db = PatternDatabase(pattern_type='corner')
db.build()  # Takes ~30 seconds
db.save('corner.db')

# Load for use
db = PatternDatabase.load('corner.db')
distance = db.lookup(state)
```

## 10. Exercises

### Exercise 1: Depth Analysis
Test how solution time grows with depth:
```python
for depth in [5, 6, 7, 8, 9, 10]:
    cube = RubikCube()
    cube.scramble(moves=depth, seed=42)
    start = time.time()
    solution = solver.solve(cube)
    elapsed = time.time() - start
    print(f"Depth {depth}: {len(solution)} moves, {elapsed:.2f}s")
    # Plot exponential growth
```

### Exercise 2: Heuristic Quality
Compare heuristic estimates vs actual distances:
```python
# For various positions:
# - Calculate h(state) from pattern databases
# - Find actual distance
# - Measure heuristic accuracy
```

### Exercise 3: Pattern Database Impact
Test with and without pattern databases:
```python
# solver_no_db = KorfSolver(use_pattern_db=False)
# solver_with_db = KorfSolver(use_pattern_db=True)
# Compare times
```

### Exercise 4: God's Number Positions
Find positions requiring exactly 20 moves:
```python
# Test many scrambles
# Find longest optimal solutions
# Verify with Korf's solver
```

## 11. References

### Original Papers

1. **Korf, R. E. (1997)** - "Finding Optimal Solutions to Rubik's Cube Using Pattern Databases"
   - Original IDA* + pattern database approach
   - Proved first optimal solutions

2. **Korf, R. E. (1985)** - "Depth-First Iterative-Deepening: An Optimal Admissible Tree Search"
   - IDA* algorithm foundation
   - Optimality proofs

3. **Rokicki, T., et al. (2010)** - "God's Number is 20"
   - Proof that 20 is maximum
   - Distribution of optimal solutions

4. **Kociemba, H., et al. (2021)** - "The Worst Position for Rubik's Cube"
   - Positions requiring 20 moves
   - Computational verification

### Implementation Resources

- Project source: `src/korf/`
- Tests: `tests/korf/`
- Benchmarks: `results/phase8_comprehensive_benchmark.json`

### Additional Reading

- Pattern databases: https://www.aaai.org/Papers/JAIR/Vol5/JAIR-514.pdf
- IDA* algorithm: https://en.wikipedia.org/wiki/Iterative_deepening_A*
- God's Number: http://www.cube20.org/

## Conclusion

Korf's IDA* algorithm represents the pinnacle of Rubik's Cube solving - **guaranteed optimal solutions**.

### Key Takeaways

1. **Optimality has a cost** - Exponential time for guaranteed shortest path
2. **Pattern databases are essential** - Make optimal solving feasible
3. **IDA* is elegant** - Memory efficient A* variant
4. **Trade-offs are fundamental** - Time vs memory vs solution quality
5. **Theoretical importance** - Proved optimal solving is possible

### When to Use Korf

**Use Korf when:**
- You need the absolute shortest solution
- Time is not critical
- Scramble is shallow (≤10 moves)
- Research or benchmarking purposes

**Don't use Korf when:**
- Speed is important
- Scramble is deep (>12 moves)
- Near-optimal is good enough
- Real-time solving needed

### The Big Picture

| Algorithm | Solution | Speed | Use Case |
|-----------|----------|-------|----------|
| Thistlethwaite | 30-52 moves | Fast | Education |
| Kociemba | ≤19 moves | Medium | Practical |
| Korf | ≤20 moves | Slow | Research |

**Kociemba wins for practical use!**

### Next Steps

- Review **05_Algorithm_Comparison.ipynb** - See all three compared
- Explore **06_Conclusion.ipynb** - Overall project summary
- Read the thesis - Full theoretical analysis

---

**Happy Cubing! 🎲**